Start compose manifest:
```
rm mr-integration/config/ml-metadata/metadata.sqlite.db && podman compose -f mr-integration-compose.yaml up
```
![](./imgs/Screenshot%202024-06-26%20at%2016.48.46.png)

In [1]:
from oml.helpers import Helper

oml = Helper() # this will connect to OCI registry

from model_registry import ModelRegistry

model_registry = ModelRegistry("localhost", 9090, author="mmortari", is_secure=False) # this will connect to Model Registry by local docker compose TODO: port once MR client rebased


/var/folders/n0/3c71vqs570b8l21fmnvb5k640000gn/T/ipykernel_64380/1045448007.py:7: UserWarning: User access token is missing
  model_registry = ModelRegistry("localhost", 9090, author="mmortari", is_secure=False) # this will connect to Model Registry by local docker compose TODO: port once MR client rebased


# Given a ML model and some metadata, to OCI registry, and then to KF Model Registry (at once)

In [2]:
# assuming a model ...
!ls -la model.joblib

# ...with some additional characteristics
accuracy_value = 0.987

-rw-r--r--@ 1 mmortari  staff  3299 Jun 25 12:23 model.joblib


![](./imgs/diagram-OCI-then-MR-2024-06-26.png)

In [3]:
from oml.dispatcher import Dispatcher

dispatcher = Dispatcher(oml, model_registry)
result = dispatcher.push_then_model_registry(
    "localhost:8080/matteo/ml-artifact:latest", "model.joblib", name="Model Example", author="John Doe", license="Apache-2.0", accuracy=accuracy_value
)

Successfully pushed localhost:8080/matteo/ml-artifact:latest
push_sha is: sha256:5a88b147bddb8f163a459b81d3605525b71fe91bb9ddf086e8377cbb43e2da9b


In [4]:
print("sha in OCI registry:", result.push_sha)

lookup_name = "Model Example"
lookup_version = result.push_sha

print("RegisteredModel:")
registered_model = model_registry.get_registered_model(lookup_name)
print(registered_model)
print("ModelVersion:")
model_version = model_registry.get_model_version(lookup_name, lookup_version)
print(model_version)
print("ModelArtifact:")
model_artifact = model_registry.get_model_artifact(lookup_name, lookup_version)
print(model_artifact)

sha in OCI registry: sha256:5a88b147bddb8f163a459b81d3605525b71fe91bb9ddf086e8377cbb43e2da9b
RegisteredModel:
RegisteredModel(id='1', description='', external_id='', create_time_since_epoch=1719415876927, last_update_time_since_epoch=1719415876927, state=<ContextState.LIVE: 'LIVE'>, name='Model Example', owner='mmortari')
ModelVersion:
ModelVersion(name='sha256:5a88b147bddb8f163a459b81d3605525b71fe91bb9ddf086e8377cbb43e2da9b', id='2', description='', external_id='', create_time_since_epoch=1719415876997, last_update_time_since_epoch=1719415876997, state=<ContextState.LIVE: 'LIVE'>, model_name='Model Example', version='sha256:5a88b147bddb8f163a459b81d3605525b71fe91bb9ddf086e8377cbb43e2da9b', author='John Doe', metadata={'accuracy': 0.987, 'license': 'Apache-2.0'}, artifacts=NOTHING, _registered_model_id=NOTHING)
ModelArtifact:
ModelArtifact(id='1', description='', external_id='', create_time_since_epoch=1719415877088, last_update_time_since_epoch=1719415877088, name='Model Example', uri

# Given some metadata entry in KF model registry, attempt retrieve pointed ML model file asset, then OCI registry

In [5]:
# assuming a model indexed on KF Model Registry ...

registeredmodel_name = "mnist"
version_name = "v0.1"

rm = model_registry.register_model(
    registeredmodel_name,
    "https://github.com/tarilabs/demo20231212/raw/main/v1.nb20231206162408/mnist.onnx",
    model_format_name="onnx",
    model_format_version="1",
    version=version_name,
    description="lorem ipsum mnist",
    metadata={
        "accuracy": 3.14,
        "license": "apache-2.0",
    }
)

![](./imgs/Screenshot%202024-06-26%20at%2017.25.10.png)

In [6]:
from oml.helpers import download_file

lookup_name = "mnist"
lookup_version = "v0.1"

registered_model = model_registry.get_registered_model(lookup_name)
model_version = model_registry.get_model_version(lookup_name, lookup_version)
model_artifact = model_registry.get_model_artifact(lookup_name, lookup_version)

file_from_mr = download_file(model_artifact.uri)

oml.push(
    f"localhost:8080/matteo/{lookup_name}:{lookup_version}",
    file_from_mr,
    name=lookup_name,
    description=model_version.description,
    author=model_version.author,
    model_format_name=model_artifact.model_format_name,
    model_format_version=model_artifact.model_format_version,
    **model_version.metadata
)

Successfully pushed localhost:8080/matteo/mnist:v0.1


<Response [201]>

![](./docs/imgs/Screenshot%202024-06-26%20at%2017.29.18.png)